# ch6 支持向量机

- 支持向量机

- 使用SMO进行优化

- 使用核函数，对数据进行空间转换

support vector machine中的“机”是因为：原始分类器，不加修改只能用于处理让二类问题，所以叫做机。

## 6-1 SVM

优点：泛化强，计算开销小；

缺点：对参数和核函数敏感。

适用：数值型，标称型

**SVM**

将数据集分割开来的直线称为分割超平面，该平面有N-1维（数据集是N维）。

数据点距离超平面越远，分割效果越好。

希望找到距离超平面最近的点（这些点又叫做支持向量），确保它们距离超平面的距离（也叫做间隔）尽可能的远。

svm优化目标：最大化支持向量到超平面的距离。

**寻找最大间隔**

超平面格式为：$w^Tx+b$，计算数据点A到该超平面的距离，即计算点到分割面的法线或垂线的长度：
$$
\frac{|w^TA+b|}{|w|}
$$

**优化问题**

使用单位跃阶函数f，令$u=w^Tx+b$，则u<0时f(u)=-1，反之则为1。

分类器接受数据输入，则输出一个类比标签-1或者1.

为什么要用-1和1，而不是0和1，是因为在计算间隔（数据点到分割超平面的距离）时，是通过$label*(w^Tx+b$计算的。

如果数据点为+1类，而且距离超平面极远，则$w^Tx+b$是极大的正数，同时$label* w^Tx+b$也是极大的正数。

如果数据点为-1类，而且距离超平面极远，则$w^Tx+b$是极大的负数，同时$label* w^Tx+b$将是极大的正数。

于是就达到了不同类别下，相同的距离度量方式。

于是优化目标变为：先寻找具有最小间隔的数据点（这些点也叫做支持向量），然后按照这些点，来寻找出w和b。

以间隔最大为优化目标，来寻找支持向量，使用以下优化函数：

$$
arg\max_{w,b}\{ \min_n(label * (w^Tx+b)) *\frac{1}{||w||} \}
$$

对于上式，令所有支持向量的$label* w^Tx+b = 1$，则距离越远的点$label* w^Tx+b$值就越大。

于是优化函数就转换为约束条件$label* w^Tx+b \leq 1 $下的优化问题：只需要计算$||w||^{-1}$最大值，来得到最终解。

于是优化目标函数变为：

$$
\max_{\alpha}[\sum_{i=1}^{m}\alpha - \frac{1}{2}\sum_{i,j=1}^{m}label^{(i)}*label^{(j)}*{\alpha}^i*{\alpha^j}<x^{(i)},y^{(i)}>]
$$

约束条件为：

$$
C \leq \alpha \leq 0,\sum_{i=1}^{m}\alpha^i*label^{(i)}=0
$$

常数C为松驰变量，控制最大化间隔和保证大部分点的函数间隔($label* (w^Tx+b)) *\frac{1}{||w||}$为函数间隔)小于1.0，这两个目标的权重。

SVM主要工作就是找到这些$\alpha$


## SMO优化算法

SMP算法目标就是求出一系列$\alpha$和b，根据$\alpha$就可以计算出权重向量w。得到超平面。

简化版SOM优化算法

In [1]:
def loadData(filename):
    # 加载数据
    dataArr=[];labelArr=[]
    fr=open(filename)
    for line in fr.readlines():
        lineArr=line.strip().split('\t')
        dataArr.append([float(lineArr[0]),float(lineArr[1])])
        labelArr.append(float(lineArr[2]))
    return dataArr,labelArr

In [2]:
import numpy as np
def selectJrand(i,m):
    # 第i个alpha的下标，m是alpha的总数
    j=i
    while (j==i):
        j=int(np.random.uniform(0,m))
    return j

In [3]:
def clipAlpha(alpha,H,L):
    # 调整大于或小于alpha的值
    if alpha>H:
        alpha=H
    if alpha<L:
        alpha=L
    return alpha

简化版的SMO算法伪代码：

```
创建alpha向量，并初始化为0向量
开始迭代：
    遍历数据集中每一个向量：
        如果该数据向量可以被优化：
            随机选择另一个数据向量
            同时优化这两个数据向量
            如果这两个数据向量都不能被优化，则退出内循环
        如果没有数据向量可被优化，则下一次迭代
```


In [4]:
import numpy as np
def smoSimple(dataArr,labelArr,C,toler,maxIter):
    # arg:数据集，标签集，松弛变量，容错率，最大迭代次数
    # return: 模型常数b,拉格朗日乘子alpha
    
    # 将数据集转为向量
    dataMat=np.mat(dataArr)
    labelMat=np.mat(labelArr).T
    # 数据维度
    m,n=np.shape(dataMat)
    # 初始化alpha和b
    b=0
    alphas=np.zeros((m,1))
    
    # 迭代
    iter=0
    # for iter in range(maxIter):
    while(iter<maxIter):
        alphaPairsChanged=0
        # 遍历数据集
        for i in range(m):
            # 预测的结果y[i]=w^T*x[i]+b，其中W=\sum_{i=1}^{m} alpha^i * label^i
            # np.multiply(m*1,m*1)=m*1,等价于(m*1)*(m*1)
            # W.T为1*m
            W=np.multiply(alphas,labelMat)
            # m*1
            # Xi=np.multiply(dataMat,dataMat[i,:].T)
            Xi=np.dot(dataMat, dataMat[i,:].T)
            print("np.shape(Xi)",np.shape(Xi))
            # 预测值
            # 1*1
            fxi=np.dot(W.T,Xi)
            fxi=np.float(fxi)
            # 计算误差
            Ei=fxi-float(labelMat[i])
            
            # 判断是否满足优化条件
            if ((labelMat[i]*Ei<-toler) and (alphas[i]<C)) or ((labelMat[i]*Ei > toler) and (alphas[i] > 0)):
                # 则随机选择另一个点
                j=selectJ(i,m)
                # 预测
                Xj=np.dot(dataMat,dataMat[j,:].T)
                # 预测值
                fxj=np.dot(W.T,Xj)
                fxj=float(fxj)
                # 计算误差
                Ej=fxj-float(labelMat[j])

                # 记录原alpha的值
                alphaIold = alphas[i].copy()
                alphaJold = alphas[j].copy()

                # L和H用于将alphas[j]调整到0-C之间。
                # 如果L==H，就不做任何改变，直接执行continue语句
                # labelMat[i] != labelMat[j] 表示异侧，就相减，否则是同侧，就相加。
                if (labelMat[i] != labelMat[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                # 如果相同，就没法优化了
                if L == H:
                    print("L==H")
                    continue
                # eta是alphas[j]的最优修改量，如果eta==0，需要退出for循环的当前迭代过程
                # 参考《统计学习方法》李航-P125~P128<序列最小最优化算法>
                eta = 2.0 * np.dot(dataMat[i, :],dataMat[j, :].T) - np.dot(dataMat[i, :],dataMat[i, :].T) - np.dot(dataMat[j, :],dataMat[j, :].T)
                if eta >= 0:
                    print("eta>=0")
                    continue
                
                # 计算出一个新的alphas[j]值
                alphas[j] =alphas[j]- labelMat[j]*(Ei - Ej)/eta
                # 并使用辅助函数，以及L和H对其进行调整
                alphas[j] = clipAlpha(alphas[j], H, L)
                # 检查alpha[j]是否只是轻微的改变，如果是的话，就退出for循环。
                if (abs(alphas[j] - alphaJold) < 0.00001):
                    print("j not moving enough")
                    continue
                
                # alphas[i]和alphas[j]同样进行改变，
                # 虽然改变的大小一样，但是改变的方向正好相反
                alphas[i] =alphas[i] + labelMat[j]*labelMat[i]*(alphaJold - alphas[j])
                # 在对alpha[i], alpha[j] 进行优化之后，给这两个alpha值设置一个常数b。
                # w= Σ[1~n] ai*yi*xi => b = yj- Σ[1~n] ai*yi(xi*xj)
                # 所以:   b1 - b = (y1-y) - Σ[1~n] yi*(a1-a)*(xi*x1)
                # 为什么减2遍？ 因为是 减去Σ[1~n]，正好2个变量i和j，所以减2遍
                b1 = b - Ei- labelMat[i]*(alphas[i]-alphaIold)*np.dot(dataMat[i, :],dataMat[i, :].T) - labelMat[j]*(alphas[j]-alphaJold)*np.dot(dataMat[i, :],dataMat[j, :].T)
                b2 = b - Ej- labelMat[i]*(alphas[i]-alphaIold)*np.dot(dataMat[i, :],dataMat[j, :].T) - labelMat[j]*(alphas[j]-alphaJold)*np.dot(dataMat[j, :],dataMat[j, :].T)
                if (0 < alphas[i]) and (C > alphas[i]):
                    b = b1
                elif (0 < alphas[j]) and (C > alphas[j]):
                    b = b2
                else:
                    b = (b1 + b2)/2.0
                alphaPairsChanged += 1
                print("iter: %d i:%d, pairs changed %d" % (iter, i, alphaPairsChanged))
        if (alphaPairsChanged == 0):
            iter += 1
        else:
            iter = 0
        print("iteration number: %d" % iter)
    return b,alphas

# test
dataArr,labelArr=loadData('./testSet.txt')
print(labelArr[:2])
b,alphas=smoSimple(dataArr,labelArr,0.6,0.001,40)

print(b)
alphas[alphas>0]

# 支持向量
for i in range(100):
    if alphas[i]>0.0:
        print(dataArr[i],labelArr[i])

## 完整版的platt SMO算法

。。。

In [5]:
import numpy as np
class optS:
    def __init__(self,dataArr,labelArr,C,toler):
        self.dataMat=np.mat(dataArr)
        self.labelMat=np.mat(labelArr)
        self.C=C
        self.tol=toler
        self.m,self.n=np.shape(dataArr)
        self.alphas=np.zeros((self.m,1))
        self.b=0
        # 是否有效，实际的E值
        self.cache=np.zeros((self.m,2))

In [ ]:
def calcEk(oS:optS,i:int):
    # 给定alpha下计算E
    # W^T
    W=np.multiply(oS.alphas,oS.labelMat)
    # m*1
    # Xi=np.multiply(dataMat,dataMat[i,:].T)
    Xi=np.dot(oS.dataMat, oS.dataMat[i,:].T)
    # 预测值 1*1
    fxi=np.dot(W.T,Xi)+oS.b
    # 计算误差
    Ei=fxi-float(oS.labelMat[i])
    # 返回误差
    return Ei

In [6]:
def selectJ(i, oS, Ei):  # this is the second choice -heurstic, and calcs Ej
    """selectJ（返回最优的j和Ej）
    内循环的启发式方法。
    选择第二个(内循环)alpha的alpha值
    这里的目标是选择合适的第二个alpha值以保证每次优化中采用最大步长。
    该函数的误差与第一个alpha值Ei和下标i有关。
    Args:
        i   具体的第i一行
        oS  optStruct对象
        Ei  预测结果与真实结果比对，计算误差Ei
    Returns:
        j  随机选出的第j一行
        Ej 预测结果与真实结果比对，计算误差Ej
    """
    maxK = -1
    maxDeltaE = 0
    Ej = 0
    # 首先将输入值Ei在缓存中设置成为有效的。这里的有效意味着它已经计算好了。
    oS.cache[i] = [1, Ei]

    # print 'oS.eCache[%s]=%s' % (i, oS.eCache[i])
    # print 'oS.eCache[:, 0].A=%s' % oS.eCache[:, 0].A.T
    # """
    # # 返回非0的: 行列值
    # nonzero(oS.eCache[:, 0].A)= (
    #     行:  array([ 0,  2,  4,  5,  8, 10, 17, 18, 20, 21, 23, 25, 26, 29, 30, 39, 46,52, 54, 55, 62, 69, 70, 76, 79, 82, 94, 97]), 
    #     列:  array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0, 0, 0])
    # )
    # """
    # print 'nonzero(oS.eCache[:, 0].A)=', nonzero(oS.eCache[:, 0].A)
    # # 取行的list
    # print 'nonzero(oS.eCache[:, 0].A)[0]=', nonzero(oS.eCache[:, 0].A)[0]
    # 非零E值的行的list列表，所对应的alpha值
    validEcacheList = np.nonzero(oS.cache[:, 0].T)[0]
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:  # 在所有的值上进行循环，并选择其中使得改变最大的那个值
            if k == i:
                continue  # don't calc for i, waste of time

            # 求 Ek误差: 预测值-真实值的差
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):
                maxK = k
                maxDeltaE = deltaE
                Ej = Ek
        return maxK, Ej
    else:  # 如果是第一次循环，则随机选择一个alpha值
        j = selectJrand(i, oS.m)

        # 求 Ek误差: 预测值-真实值的差
        Ej = calcEk(oS, j)
    return j, Ej

In [7]:
def updateEk(oS, k):  # after any alpha has changed update the new value in the cache
    # 计算误差并存入缓存
    """updateEk（计算误差值并存入缓存中。）
    在对alpha值进行优化之后会用到这个值。
    Args:
        oS  optStruct对象
        k   某一列的行号
    """
    # 求 误差: 预测值-真实值的差
    Ek = calcEk(oS, k)
    oS.cache[k] = [1, Ek]

In [8]:
def innerL(i, oS):
    """innerL
    内循环代码
    Args:
        i   具体的某一行
        oS  optStruct对象
    Returns:
        0   找不到最优的值
        1   找到了最优的值，并且oS.Cache到缓存中
    """

    # 求 Ek误差: 预测值-真实值的差
    Ei = calcEk(oS, i)

    # 约束条件 (KKT条件是解决最优化问题的时用到的一种方法。我们这里提到的最优化问题通常是指对于给定的某一函数，求其在指定作用域上的全局最小值)
    # 0<=alphas[i]<=C，但由于0和C是边界值，我们无法进行优化，因为需要增加一个alphas和降低一个alphas。
    # 表示发生错误的概率: labelMat[i]*Ei 如果超出了 toler， 才需要优化。至于正负号，我们考虑绝对值就对了。
    if ((oS.labelMat[i] * Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i] * Ei > oS.tol) and (oS.alphas[i] > 0)):
        # 选择最大的误差对应的j进行优化。效果更明显
        j, Ej = selectJ(i, oS, Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()

        # L和H用于将alphas[j]调整到0-C之间。如果L==H，就不做任何改变，直接return 0
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L == H:
            print("L==H")
            return 0

        # eta是alphas[j]的最优修改量，如果eta==0，需要退出for循环的当前迭代过程
        # 参考《统计学习方法》李航-P125~P128<序列最小最优化算法>
        eta = 2.0 * oS.dataMat[i, :] * oS.dataMat[j, :].T - oS.dataMat[i, :] * oS.dataMat[i, :].T - oS.dataMat[j, :] * oS.dataMat[j, :].T
        if eta >= 0:
            print("eta>=0")
            return 0

        # 计算出一个新的alphas[j]值
        oS.alphas[j] =oS.alphas[j]- oS.labelMat[j] * (Ei - Ej) / eta
        # 并使用辅助函数，以及L和H对其进行调整
        oS.alphas[j] = clipAlpha(oS.alphas[j], H, L)
        # 更新误差缓存
        updateEk(oS, j)

        # 检查alpha[j]是否只是轻微的改变，如果是的话，就退出for循环。
        if (abs(oS.alphas[j] - alphaJold) < 0.00001):
            print("j not moving enough")
            return 0

        # 然后alphas[i]和alphas[j]同样进行改变，虽然改变的大小一样，但是改变的方向正好相反
        oS.alphas[i] =oS.alphas[i] + oS.labelMat[j] * oS.labelMat[i] * (alphaJold - oS.alphas[j])
        # 更新误差缓存
        updateEk(oS, i)

        # 在对alpha[i], alpha[j] 进行优化之后，给这两个alpha值设置一个常数b。
        # w= Σ[1~n] ai*yi*xi => b = yj Σ[1~n] ai*yi(xi*xj)
        # 所以:   b1 - b = (y1-y) - Σ[1~n] yi*(a1-a)*(xi*x1)
        # 为什么减2遍？ 因为是 减去Σ[1~n]，正好2个变量i和j，所以减2遍
        b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.dataMat[i, :] * oS.dataMat[i, :].T - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.dataMat[i, :] * oS.dataMat[j, :].T
        b2 = oS.b - Ej - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.dataMat[i, :] * oS.dataMat[j, :].T - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.dataMat[j, :] * oS.dataMat[j, :].T
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]):
            oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]):
            oS.b = b2
        else:
            oS.b = (b1 + b2) / 2.0
        return 1
    else:
        return 0


In [9]:
def smoP(dataMatIn, classLabels, C, toler, maxIter):
    """
    完整SMO算法外循环，与smoSimple有些类似，但这里的循环退出条件更多一些
    Args:
        dataMatIn    数据集
        classLabels  类别标签
        C   松弛变量(常量值)，允许有些数据点可以处于分隔面的错误一侧。
            控制最大化间隔和保证大部分的函数间隔小于1.0这两个目标的权重。
            可以通过调节该参数达到不同的结果。
        toler   容错率
        maxIter 退出前最大的循环次数
    Returns:
        b       模型的常量值
        alphas  拉格朗日乘子
    """

    # 创建一个 optStruct 对象
    oS = optStruct(np.mat(dataMatIn), np.mat(classLabels).transpose(), C, toler)
    iter = 0
    entireSet = True
    alphaPairsChanged = 0

    # 循环遍历: 循环maxIter次 并且 （alphaPairsChanged存在可以改变 or 所有行遍历一遍）
    # 循环迭代结束 或者 循环遍历所有alpha后，alphaPairs还是没变化
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0

        #  当entireSet=true or 非边界alpha对没有了；就开始寻找 alpha对，然后决定是否要进行else。
        if entireSet:
            # 在数据集上遍历所有可能的alpha
            for i in range(oS.m):
                # 是否存在alpha对，存在就+1
                alphaPairsChanged += innerL(i, oS)
                print("fullSet, iter: %d i:%d, pairs changed %d" % (iter, i, alphaPairsChanged))
            iter += 1
        # 对已存在 alpha对，选出非边界的alpha值，进行优化。
        else:
            # 遍历所有的非边界alpha值，也就是不在边界0或C上的值。
            nonBoundIs = np.nonzero((oS.alphas.T > 0) * (oS.alphas.T < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i, oS)
                print("non-bound, iter: %d i:%d, pairs changed %d" % (iter, i, alphaPairsChanged))
            iter += 1

        # 如果找到alpha对，就优化非边界alpha值，否则，就重新进行寻找，如果寻找一遍 遍历所有的行还是没找到，就退出循环。
        if entireSet:
            entireSet = False  # toggle entire set loop
        elif (alphaPairsChanged == 0):
            entireSet = True
        print("iteration number: %d" % iter)
    return oS.b, oS.alphas

In [10]:
# test
dataArr,labelArr=loadData("./testSet.txt")
b,alphas=smoP(dataArr,labelArr,0.6,0.001,40)

L==H
fullSet, iter: 0 i:0, pairs changed 0
L==H
fullSet, iter: 0 i:1, pairs changed 0
fullSet, iter: 0 i:2, pairs changed 1
L==H
fullSet, iter: 0 i:3, pairs changed 1
fullSet, iter: 0 i:4, pairs changed 2
fullSet, iter: 0 i:5, pairs changed 2
fullSet, iter: 0 i:6, pairs changed 2
j not moving enough
fullSet, iter: 0 i:7, pairs changed 2
L==H
fullSet, iter: 0 i:8, pairs changed 2
fullSet, iter: 0 i:9, pairs changed 2
L==H
fullSet, iter: 0 i:10, pairs changed 2
L==H
fullSet, iter: 0 i:11, pairs changed 2
L==H
fullSet, iter: 0 i:12, pairs changed 2
fullSet, iter: 0 i:13, pairs changed 2
L==H
fullSet, iter: 0 i:14, pairs changed 2
fullSet, iter: 0 i:15, pairs changed 2
fullSet, iter: 0 i:16, pairs changed 2
L==H
fullSet, iter: 0 i:17, pairs changed 2
fullSet, iter: 0 i:18, pairs changed 3
fullSet, iter: 0 i:19, pairs changed 3
fullSet, iter: 0 i:20, pairs changed 3
fullSet, iter: 0 i:21, pairs changed 3
j not moving enough
fullSet, iter: 0 i:22, pairs changed 3
L==H
fullSet, iter: 0 i:23, 

计算得到alpha向量之后，就可以得到超平面(alpha[i]>0.0的第i个数据点构成支持向量)，

In [16]:
def calcWs(alphas, dataArr, classLabels):
    """
    基于alpha计算w值
    Args:
        alphas        拉格朗日乘子
        dataArr       feature数据集
        classLabels   目标变量数据集
    Returns:
        wc  回归系数
    """
    X = np.mat(dataArr)
    labelMat = np.mat(classLabels).T
    m, n = np.shape(X)
    w = np.zeros((n, 1))
    for i in range(m):
        w =w+ np.multiply(alphas[i] * labelMat[i], X[i, :].T)
    return w

In [17]:
ws=calcWs(alphas,dataArr,labelArr)
ws

matrix([[ 0.65307162],
        [-0.17196128]])

In [21]:
# 对第一个数据点分类
dataMat=np.mat(dataArr)
y=dataMat[0]*np.mat(ws)+b
print(y)
print(labelArr[0])

[[-0.92555695]]
-1.0


以下开始，清除所有变量。从新开始

## 核函数的SVM

以下这些辅助函数不发生变化

In [1]:
def loadData(filename):
    # 加载数据
    dataArr=[];labelArr=[]
    fr=open(filename)
    for line in fr.readlines():
        lineArr=line.strip().split('\t')
        dataArr.append([float(lineArr[0]),float(lineArr[1])])
        labelArr.append(float(lineArr[2]))
    return dataArr,labelArr

In [2]:
import numpy as np
def selectJrand(i,m):
    # 第i个alpha的下标，m是alpha的总数
    j=i
    while (j==i):
        j=int(np.random.uniform(0,m))
    return j

In [3]:
def clipAlpha(alpha,H,L):
    # 调整大于或小于alpha的值
    if alpha>H:
        alpha=H
    if alpha<L:
        alpha=L
    return alpha

In [4]:
def selectJ(i, oS, Ei):  # this is the second choice -heurstic, and calcs Ej
    """selectJ（返回最优的j和Ej）
    内循环的启发式方法。
    选择第二个(内循环)alpha的alpha值
    这里的目标是选择合适的第二个alpha值以保证每次优化中采用最大步长。
    该函数的误差与第一个alpha值Ei和下标i有关。
    Args:
        i   具体的第i一行
        oS  optStruct对象
        Ei  预测结果与真实结果比对，计算误差Ei
    Returns:
        j  随机选出的第j一行
        Ej 预测结果与真实结果比对，计算误差Ej
    """
    maxK = -1
    maxDeltaE = 0
    Ej = 0
    # 首先将输入值Ei在缓存中设置成为有效的。这里的有效意味着它已经计算好了。
    oS.cache[i] = [1, Ei]

    # print 'oS.eCache[%s]=%s' % (i, oS.eCache[i])
    # print 'oS.eCache[:, 0].A=%s' % oS.eCache[:, 0].A.T
    # """
    # # 返回非0的: 行列值
    # nonzero(oS.eCache[:, 0].A)= (
    #     行:  array([ 0,  2,  4,  5,  8, 10, 17, 18, 20, 21, 23, 25, 26, 29, 30, 39, 46,52, 54, 55, 62, 69, 70, 76, 79, 82, 94, 97]), 
    #     列:  array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,0, 0, 0, 0, 0])
    # )
    # """
    # print 'nonzero(oS.eCache[:, 0].A)=', nonzero(oS.eCache[:, 0].A)
    # # 取行的list
    # print 'nonzero(oS.eCache[:, 0].A)[0]=', nonzero(oS.eCache[:, 0].A)[0]
    # 非零E值的行的list列表，所对应的alpha值
    validEcacheList = np.nonzero(oS.cache[:, 0].T)[0]
    if (len(validEcacheList)) > 1:
        for k in validEcacheList:  # 在所有的值上进行循环，并选择其中使得改变最大的那个值
            if k == i:
                continue  # don't calc for i, waste of time

            # 求 Ek误差: 预测值-真实值的差
            Ek = calcEk(oS, k)
            deltaE = abs(Ei - Ek)
            if (deltaE > maxDeltaE):
                maxK = k
                maxDeltaE = deltaE
                Ej = Ek
        return maxK, Ej
    else:  # 如果是第一次循环，则随机选择一个alpha值
        j = selectJrand(i, oS.m)

        # 求 Ek误差: 预测值-真实值的差
        Ej = calcEk(oS, j)
    return j, Ej

In [5]:
def updateEk(oS, k):  # after any alpha has changed update the new value in the cache
    # 计算误差并存入缓存
    """updateEk（计算误差值并存入缓存中。）
    在对alpha值进行优化之后会用到这个值。
    Args:
        oS  optStruct对象
        k   某一列的行号
    """
    # 求 误差: 预测值-真实值的差
    Ek = calcEk(oS, k)
    oS.cache[k] = [1, Ek]

In [68]:
def smoP(dataMatIn, classLabels, C, toler, maxIter,ktup):
    """
    完整SMO算法外循环，与smoSimple有些类似，但这里的循环退出条件更多一些
    Args:
        dataMatIn    数据集
        classLabels  类别标签
        C   松弛变量(常量值)，允许有些数据点可以处于分隔面的错误一侧。
            控制最大化间隔和保证大部分的函数间隔小于1.0这两个目标的权重。
            可以通过调节该参数达到不同的结果。
        toler   容错率
        maxIter 退出前最大的循环次数
    Returns:
        b       模型的常量值
        alphas  拉格朗日乘子
    """

    # 创建一个 optStruct 对象
    oS = optStruct(np.mat(dataMatIn), np.mat(classLabels).transpose(), C, toler,ktup)
    iter = 0
    entireSet = True
    alphaPairsChanged = 0

    # 循环遍历: 循环maxIter次 并且 （alphaPairsChanged存在可以改变 or 所有行遍历一遍）
    # 循环迭代结束 或者 循环遍历所有alpha后，alphaPairs还是没变化
    while (iter < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0

        #  当entireSet=true or 非边界alpha对没有了；就开始寻找 alpha对，然后决定是否要进行else。
        if entireSet:
            # 在数据集上遍历所有可能的alpha
            for i in range(oS.m):
                # 是否存在alpha对，存在就+1
                alphaPairsChanged += innerL(i, oS)
                # print("fullSet, iter: %d i:%d, pairs changed %d" % (iter, i, alphaPairsChanged))
            iter += 1
        # 对已存在 alpha对，选出非边界的alpha值，进行优化。
        else:
            # 遍历所有的非边界alpha值，也就是不在边界0或C上的值。
            nonBoundIs = np.nonzero((oS.alphas.T > 0) * (oS.alphas.T < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i, oS)
                # print("non-bound, iter: %d i:%d, pairs changed %d" % (iter, i, alphaPairsChanged))
            iter += 1

        # 如果找到alpha对，就优化非边界alpha值，否则，就重新进行寻找，如果寻找一遍 遍历所有的行还是没找到，就退出循环。
        if entireSet:
            entireSet = False  # toggle entire set loop
        elif (alphaPairsChanged == 0):
            entireSet = True
        print("iteration number: %d" % iter)
    return oS.b, oS.alphas

In [7]:
def calcWs(alphas, dataArr, classLabels):
    """
    基于alpha计算w值
    Args:
        alphas        拉格朗日乘子
        dataArr       feature数据集
        classLabels   目标变量数据集
    Returns:
        wc  回归系数
    """
    X = np.mat(dataArr)
    labelMat = np.mat(classLabels).T
    m, n = np.shape(X)
    w = np.zeros((n, 1))
    for i in range(m):
        w =w+ np.multiply(alphas[i] * labelMat[i], X[i, :].T)
    return w

## 在复杂数据上应用核函数

核函数，将数据转换为易于分类器理解的形式。

通过核函数，将数据集从一个特征空间映射另一个特征空间。

可以将核函数想象为包装器或者接口，它把数据转换为一种容易处理的形式。

## 径向基核函数

该函数以向量为自变量，计算出一个标量。这里使用径向基函数的高斯版本：

$$
k(x,y)=exp(\frac{-||x-y||^2}{2{\sigma}^2})
$$
其中，$\sigma$是用于确定到达率（或函数值跌落带0的速度参数）。

高斯核函数将数据映射到更高维的空间。

In [8]:
def kernelTrans(dataArr,dataLine,ktup):
    m,n=np.shape(dataArr)
    kern=np.zeros((m,1))
    if ktup[0]=='lin':
        kern=np.dot(dataArr,dataLine.T)
    elif ktup[0]=='rbf':
        for j in range(m):
            dis=dataArr[j,:]-dataLine
            kern[j]=np.dot(dis,dis.T)
        kern=np.exp(kern/(-1 * ktup[1]**2))
    else:
        print("ktup error")
        return 
    return kern

## 将核转换函数，加入到SMO算法中

In [64]:
class optStruct:
    def __init__(self,dataArr,labelArr,C,toler,ktup):
        # 相比无kernel的原SMO，此处多了一个参数
        self.dataMat=np.mat(dataArr)
        self.labelMat=np.mat(labelArr)
        self.C=C
        self.tol=toler
        self.m,self.n=np.shape(dataArr)
        self.alphas=np.zeros((self.m,1))
        self.b=0
        # 是否有效，实际的E值
        self.cache=np.zeros((self.m,2))
        # 新增
        self.K=np.zeros( (self.m,self.m) )
        for i in range(self.m):
            self.K[:,i]=kernelTrans(self.dataMat,self.dataMat[i,:],ktup).reshape(self.m)

加入核函数之后，innerL函数和calcEK函数需要做一些修改

In [71]:
def innerL(i, oS):
    """innerL
    内循环代码
    Args:
        i   具体的某一行
        oS  optStruct对象
    Returns:
        0   找不到最优的值
        1   找到了最优的值，并且oS.Cache到缓存中
    """

    # 求 Ek误差: 预测值-真实值的差
    Ei = calcEk(oS, i)

    if ((oS.labelMat[i] * Ei < -oS.tol) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i] * Ei > oS.tol) and (oS.alphas[i] > 0)):
        # 选择最大的误差对应的j进行优化。效果更明显
        j, Ej = selectJ(i, oS, Ei)
        alphaIold = oS.alphas[i].copy()
        alphaJold = oS.alphas[j].copy()

        # L和H用于将alphas[j]调整到0-C之间。如果L==H，就不做任何改变，直接return 0
        if (oS.labelMat[i] != oS.labelMat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L == H:
            return 0

        # eta是alphas[j]的最优修改量，如果eta==0，需要退出for循环的当前迭代过程
        # 修改
        eta = 2.0 * oS.K[i,j] * oS.K[i, i] - oS.K[j,j]

        if eta >= 0:
            return 0

        # 计算出一个新的alphas[j]值
        oS.alphas[j] =oS.alphas[j]- oS.labelMat[j] * (Ei - Ej) / eta
        # 并使用辅助函数，以及L和H对其进行调整
        oS.alphas[j] = clipAlpha(oS.alphas[j], H, L)
        # 更新误差缓存
        updateEk(oS, j)

        # 检查alpha[j]是否只是轻微的改变，如果是的话，就退出for循环。
        if (abs(oS.alphas[j] - alphaJold) < 0.00001):
            return 0

        # 然后alphas[i]和alphas[j]同样进行改变，虽然改变的大小一样，但是改变的方向正好相反
        oS.alphas[i] =oS.alphas[i] + oS.labelMat[j] * oS.labelMat[i] * (alphaJold - oS.alphas[j])
        # 更新误差缓存
        updateEk(oS, i)

        # 在对alpha[i], alpha[j] 进行优化之后，给这两个alpha值设置一个常数b。
        # 修改
        b1 = oS.b - Ei - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.K[i,i] - oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.K[i,j]
        b2 = oS.b - Ej - oS.labelMat[i] * (oS.alphas[i] - alphaIold) * oS.K[i,j]- oS.labelMat[j] * (oS.alphas[j] - alphaJold) * oS.K[j,j]
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]):
            oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]):
            oS.b = b2
        else:
            oS.b = (b1 + b2) / 2.0
        return 1
    else:
        return 0

In [72]:
def calcEk(oS:optStruct,i:int):
    # 给定alpha下计算E
    # W^T
    W=np.multiply(oS.alphas,oS.labelMat)
    # m*1
    # Xi=np.multiply(dataMat,dataMat[i,:].T)
    Xi=oS.K[:,i]
    # 预测值 1*1
    fxi=np.dot(W.T,Xi)+oS.b
    # 计算误差
    Ei=fxi-float(oS.labelMat[i])
    # 返回误差
    return Ei

测试 

# test
dataArr,labelArr=loadData("./testSet.txt")
b,alphas=smoP(dataArr,labelArr,200,0.0001,10000,('rbf',1.3))

In [73]:
def modelTest():
    dataArr,labelArr=loadData("./testSetRBF.txt")
    b,alphas=smoP(dataArr,labelArr,200,0.0001,10000,('rbf',1.3))
    dataMat=np.mat(dataArr);labelMat=np.mat(labelArr).T
    svIndex=np.nonzero(alphas)[0]
    svS=dataMat[svIndex]
    labelSv=labelMat[svIndex]
    m,n=np.shape(dataMat)
    errCnt=0
    for i in range(m):
        kern=kernelTrans(svS,dataMat[i,:],('rbf',1.3))
        predict=np.dot(kern.T,np.multiply(labelSv,alphas[svIndex]))+b
        if np.sign(predict)!=np.sign(labelArr[i]):
            errCnt+=1
    return 1-errCnt/float(m)

In [74]:
modelTest()

iteration number: 1
iteration number: 2
iteration number: 3
iteration number: 4
iteration number: 5
iteration number: 6
iteration number: 7


0.6599999999999999

正确率相差太大，貌似那里出了bug。

先跳过了。